In [50]:
# Importar librerias y lenguajes
import spacy 
from spacy.tokens import Token, Span
# nlp = spacy.blank("es") # crea un pipeline vacio para procesar el espanol.
nlp = spacy.load("es_core_news_sm") # Descarga el paquete del lenguaje espanol preentrenado para poder trabajar con las capacidades centrales.


In [51]:
# Crear el texto, en un objeto llamado "doc".
doc = nlp("Arroz con leche, me quiero casar"
        " Con uno señora de San Nicolás"
        " Que sepa cocer, que sepa bordar"
        " Que sepa abrir la puerta para ir a jugar"
        " Yo soy la viudita del barrio del rey"
        " Me quiero casar y no sé con quién"
        " Con esta sí, con esta no"
        " Con esta señorita me caso yo"
        " Arroz con leche, me quiero casar"
        " Con una señorita de San Nicolás"
        " Que sepa cocer, que sepa bordar"
        " Que sepa abrir la puerta para ir a jugar"
        " Yo soy la viudita del barrio del rey"
        " Me quiero casar y no sé con quién"
        " Con esta sí, con esta no"
        " Con esta señorita me caso yo")

# Iterar sobre los tokens en el doc. Spacy transforma cada palabra, signo, puntuacion del doc en un token.
for token in doc:
    print(token.text)

Arroz
con
leche
,
me
quiero
casar
Con
uno
señora
de
San
Nicolás
Que
sepa
cocer
,
que
sepa
bordar
Que
sepa
abrir
la
puerta
para
ir
a
jugar
Yo
soy
la
viudita
del
barrio
del
rey
Me
quiero
casar
y
no
sé
con
quién
Con
esta
sí
,
con
esta
no
Con
esta
señorita
me
caso
yo
Arroz
con
leche
,
me
quiero
casar
Con
una
señorita
de
San
Nicolás
Que
sepa
cocer
,
que
sepa
bordar
Que
sepa
abrir
la
puerta
para
ir
a
jugar
Yo
soy
la
viudita
del
barrio
del
rey
Me
quiero
casar
y
no
sé
con
quién
Con
esta
sí
,
con
esta
no
Con
esta
señorita
me
caso
yo


In [52]:
# El objeto doc tiene su propio indice para el acceso directo.
token = doc[5]
print(token.text) # la funcion .text devuelve el texto.
print("---------------------")
# Un objeto SPAN es un slice del doc, compuesto por 1 o mas tokens. Es un view, no contiene los datos en si.
span = doc[4:13]
print(span)
print("---------------------")
print("Índice:   ", [token.i for token in doc[0:10]])
print("---------------------")
# Reconocimiento de numeros, puntuaciones y palabras alfabeticas (llamados tambien atributos lexicos, no dependen del contexto):
print("is_alpha:", [token.is_alpha for token in doc[0:10]])
print("is_punct:", [token.is_punct for token in doc[0:10]])
print("like_num:", [token.like_num for token in doc[0:10]])

quiero
---------------------
me quiero casar Con uno señora de San Nicolás
---------------------
Índice:    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
---------------------
is_alpha: [True, True, True, False, True, True, True, True, True, True]
is_punct: [False, False, False, True, False, False, False, False, False, False]
like_num: [False, False, False, False, False, False, False, False, True, False]


In [53]:
# Itera sobre los tokens en el doc
for token in doc:
    # Revisa si el token parece un número
    if token.like_num:
        # Obtén el próximo token en el documento
        next_token = doc[token.i + 1]
        # Revisa si el texto del siguiente token es igual a '%'
        if next_token.text == "señorita":
            print("Token encontrado:", (token.text))

In [55]:
# Componentes del pipeline entrenados.

# Parser (Etiquetas gramaticales)
for token in doc[0:10]:
    # Imprime en pantalla el texto y la etiqueta gramatical predicha
    print(token.text, token.pos_) # En spaCy, los atributos que devuelven un string normalmente terminan con un guion bajo (_). 
                                  # Mientras que atributos sin un guion bajo devuelven un valor ID de tipo entero.

Arroz PROPN
con ADP
leche NOUN
, PUNCT
me PRON
quiero VERB
casar VERB
Con ADP
uno NUM
señora NOUN


In [56]:
# Tagger (Relaciones entre las palabras)
for token in doc[0:10]:
    print(token.text, token.dep_, token.head.text) # El atributo .dep_ devuelve la etiqueta de la dependencia sintáctica predicha.
                                                               # El atributo .head devuelve el token de la cabeza de la dependencia sintáctica.

Arroz ROOT Arroz
con case leche
leche nmod Arroz
, punct Arroz
me iobj quiero
quiero ROOT quiero
casar xcomp quiero
Con case señora
uno det señora
señora obl casar


In [60]:
#Entity recognizer (entidades nombradas: objetos de la vida real que tienen un nombre asignado)

for ent in doc.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(ent.text, ent.label_) # La propiedad doc.ents te permite acceder a las entidades nombradas predichas por el modelo de reconocimiento de entidades.
                                # La propiedad doc.ents devuelve un iterador de objetos Span, 
                                # así que podemos imprimir en pantalla el texto y la etiqueta de la entidad usando el atributo .label_.

spacy.explain("MISC") # Explica las definiciones de las etiquetas.
# Mal resultado.

Con uno señora de San Nicolás Que sepa cocer MISC
Con esta sí MISC
Con esta señorita me caso yo Arroz MISC
Con una señorita de San Nicolás Que sepa cocer MISC
Con esta sí MISC
Con esta señorita me caso yo MISC


'Miscellaneous entities, e.g. events, nationalities, products or works of art'